In [177]:
import os
import utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from functools import partial, reduce

In [178]:
def read_all_results(dir_path):
    paths = os.listdir(dir_path)
    last_gens = []
    for i in range(len(paths)):
        path = dir_path + paths[i]
        try:
            last_gen = sorted(map(lambda x: x.split("_"), 
                            os.listdir(f"{path}/individuals/")), 
                            key=lambda x: int(x[1].split(".")[0]))[-1][-1][:-4]
            print(f"{i+1}: Path: {paths[i]}, last generation: {int(last_gen)}")
            last_gens.append(int(last_gen))
        except:
            continue
    return paths, last_gens

In [179]:
dir_path = "results/"
paths, last_gens = read_all_results(dir_path)

1: Path: GA_2021_06_05_13_19_14, last generation: 122
2: Path: GA_2021_06_05_13_18_39, last generation: 122
3: Path: GA_2021_06_05_17_50_02, last generation: 109
4: Path: GA_2021_06_06_12_19_20, last generation: 75
5: Path: GA_2021_06_05_13_19_08, last generation: 121
6: Path: GA_2021_06_05_16_10_38, last generation: 114
7: Path: GA_2021_06_06_12_18_40, last generation: 81
8: Path: GA_2021_06_05_13_18_32, last generation: 122


In [272]:
run = 8
path = dir_path + paths[run-1]
with open(path + "/run_params.json", "r") as file:
    print(file.read())
    file.close()

{
    "objective": "weighted",
    "simulations": 30,
    "process": {
        "horizon": 37,
        "decision_period": 56,
        "policy": "weighted"
    },
    "population_size": 30,
    "random_individuals": false,
    "min_generations": 200
}


## Individual results

In [273]:
gen = 122
individuals_path =  f"{path}/individuals/individuals_{gen}.pkl"
individuals = utils.read_pickle(individuals_path)

In [274]:
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])
print("Strategy_count:")
print(f"{utils.tcolors.BOLD}{'#':<3} {'U':<5}{'D':<5}{'N':<5}{utils.tcolors.ENDC}")
for trend_count in range(6):
    strategy_count = individuals[0].strategy_count
    print(f"{trend_count+1:<2}| {strategy_count['U'][trend_count]:<5}{strategy_count['D'][trend_count]:<5}{strategy_count['N'][trend_count]:<5}")
print()
for ix, i in enumerate(sorted(individuals, key=lambda x: x.mean_score)):
    print(f"{utils.tcolors.BOLD}{ordinal(ix+1)} place:{utils.tcolors.ENDC}")
    print(f"ID: {i.ID}")
    print(f"Score: {np.round(i.mean_score, 3)}")
    print(f"Genes:\n{np.round(i.genes, 3)}")
    print()

Strategy_count:
#   U    D    N    
1 | 28   30   30   
2 | 24   25   30   
3 | 15   12   30   
4 | 3    5    27   
5 | 0    0    14   
6 | 0    0    4    

1st place:
ID: gen_105_015
Score: 2119.035
Genes:
[[[0.    0.004 0.964 0.028 0.004]
  [0.    0.015 0.951 0.03  0.004]
  [0.    0.015 0.951 0.03  0.004]]

 [[0.094 0.039 0.823 0.042 0.002]
  [0.    0.015 0.951 0.03  0.004]
  [0.    0.04  0.901 0.049 0.011]]

 [[0.    0.047 0.9   0.049 0.004]
  [0.    0.017 0.949 0.03  0.004]
  [0.    0.015 0.951 0.03  0.004]]]

2nd place:
ID: gen_82_011
Score: 2169.27
Genes:
[[[0.    0.    0.914 0.086 0.   ]
  [0.    0.    0.914 0.086 0.   ]
  [0.    0.    0.477 0.523 0.   ]]

 [[0.    0.    0.914 0.086 0.   ]
  [0.    0.    0.914 0.086 0.   ]
  [0.    0.    0.911 0.089 0.   ]]

 [[0.    0.    0.914 0.086 0.   ]
  [0.    0.    0.914 0.086 0.   ]
  [0.    0.    0.911 0.089 0.   ]]]

3rd place:
ID: gen_118_012
Score: 2174.531
Genes:
[[[0.    0.    0.959 0.041 0.   ]
  [0.    0.    0.957 0.043 0.   ]
 

In [275]:
genes = np.sum(individuals[0].genes, axis=(0,1))
norm = np.sum(genes)
norm_genes = np.divide(genes, norm)
print(norm_genes)

[0.01041667 0.02301263 0.92685053 0.03532578 0.0043944 ]
